In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path.append("../src")
import os
from src.DS_Generator import DataSet_Generator
from src.model_evaluation import mean_average_precision
from data_augmentation import *
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from util import *
import datetime
%load_ext tensorboard

In [2]:
train_ds, val_ds = DataSet_Generator().generate_species_data(TRAIN_SPECIES_DF,augment=1,batch_size=32)
num_classes = len(set(TRAIN_SPECIES_DF["species"]))

We threw away the datapoint with index 380 


In [3]:
Input = tf.keras.Input((224,224,3))
base = tf.keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False,input_tensor=Input)

flatten = base.output
flatten = tf.keras.layers.Flatten()(flatten)
head = tf.keras.layers.Dense(256, activation="relu")(flatten)
head = tf.keras.layers.Dense(128, activation="relu")(head)
head = tf.keras.layers.Dense(64, activation="relu")(head)
head = tf.keras.layers.Dense(num_classes,activation="softmax")(head)
model = tf.keras.Model(inputs=Input, outputs=head,name="Resnet50imagenetweightssoftmax")

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss="categorical_crossentropy",metrics=["acc"])

In [5]:
if model.name not in os.listdir("../../models/"):
    os.makedirs("../../models/"+model.name)
    os.makedirs("../../models/"+model.name+"/logs")
    os.makedirs("../../models/"+model.name+"/saves")
    
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = "../../models/"+model.name+"/logs/" +time_stamp 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "../../models/"+model.name+"/saves/" +time_stamp +"/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [ ]:
model.fit(
train_ds,
epochs=15,
validation_data=val_ds,
callbacks=[cp_callback,tensorboard_callback])

Epoch 1/15
1436/1436 [==============================] - ETA: 0s - loss: 1.4678 - acc: 0.6030
Epoch 1: saving model to ../../models/Resnet50imagenetweightssoftmax/saves/20220328-153544\cp-0001.ckpt
1436/1436 [==============================] - 513s 351ms/step - loss: 1.4678 - acc: 0.6030 - val_loss: 1.3748 - val_acc: 0.5963
Epoch 2/15
1436/1436 [==============================] - ETA: 0s - loss: 1.1364 - acc: 0.6688
Epoch 2: saving model to ../../models/Resnet50imagenetweightssoftmax/saves/20220328-153544\cp-0002.ckpt
1436/1436 [==============================] - 504s 349ms/step - loss: 1.1364 - acc: 0.6688 - val_loss: 1.1109 - val_acc: 0.6798
Epoch 3/15
1436/1436 [==============================] - ETA: 0s - loss: 0.8340 - acc: 0.7514
Epoch 3: saving model to ../../models/Resnet50imagenetweightssoftmax/saves/20220328-153544\cp-0003.ckpt
1436/1436 [==============================] - 504s 349ms/step - loss: 0.8340 - acc: 0.7514 - val_loss: 0.8254 - val_acc: 0.7461
Epoch 4/15
1436/1436 [======

In [ ]:
5+5